# Virtual Scribe AI Comparator

The goal of this notebook is to compare the ability of various AI models to generate a Geriatrics Consultation Note

In [1]:
# Load necessary components
import os
import json

from langchain.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


In [2]:
# Function to read notes andput into array
def read_notes_to_array(ehr):
    notes = []
    for filename in os.listdir(ehr):
        note_path = os.path.join(ehr,filename)

        if os.path.isfile(note_path) and filename.endswith(".txt"):
            with open(note_path, 'r') as note:
                notes.append(note.read())

    return notes

# Function to pull template from file
def template_generation(template):
    with open("templates/"+template+".txt", "r") as f:
        template = f.read()
    return template

In [3]:
# Pick models to test
models = [
    "llama3.2",
    "llama3",
    "qwen2",
    "phi3",
    "gemma2",
    "medllama3-v20",
    "meditron",
    "medllama2",
    "MedX3.2",
    "NewMes15"
]

# Get notes
notes = read_notes_to_array("ehr")

In [4]:
# Create the prompt
prompt = PromptTemplate(
    input_variables=["notes"],
    template=template_generation("geriatric_consult")+"Use the following information to fill out the above template: {notes}"
)

In [5]:
for model in models:
    llm = Ollama(model=model)
    i = 0
    for note in notes:
        i += 1
        with open("virtualscribe_output/"+model+"patient-"+str(i)+".txt", "w") as f:
            f.write(json.dumps(LLMChain(llm=llm,prompt=prompt).invoke(note)).replace("\\n", "\n"))

/var/folders/44/xj8x8w150zn_bwb5z9fcs9dw0000gn/T/ipykernel_4338/247565835.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model=model)
/var/folders/44/xj8x8w150zn_bwb5z9fcs9dw0000gn/T/ipykernel_4338/247565835.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  f.write(json.dumps(LLMChain(llm=llm,prompt=prompt).invoke(note)).replace("\\n", "\n"))


OllamaEndpointNotFoundError: Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull llama3.2`.